In [ ]:
from keras.layers import GlobalAvgPool2D
from keras.layers import MaxPool2D
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import DepthwiseConv2D
from keras.layers import BatchNormalization
from keras.layers import ReLU
from keras.layers import Dense
from keras.layers import Add
from keras.layers.merge import concatenate

In [ ]:
input_layer = Input(shape=(224,224, 3))
model = Create_Resnet50_Model(input_layer,1000)
model.summary()
# =plot_model(model, show_shapes=True, to_file='resnet_module.png')

In [ ]:
def Create_Resnet50_Model(input_layer,number_classes):
  
  layer1 = Conv2D(64,(7,7),strides = (2,2) , padding='same')(input_layer)
  Layer1_norm = BatchNormalization()(layer1)
  layer1_relu = ReLU()(Layer1_norm)

  maxpool = MaxPool2D((3,3), strides=(2,2),padding='same' )(layer1_relu)
  # print(conv_2)
  #conv_2 x3
  conv_2 = resnet_conv_layer(maxpool,64,2)
  for i in range(2):
    conv_2 = resnet_layer(conv_2 , 64)

  #conv_3 x4
  conv_3 = resnet_conv_layer(conv_2,128,2)
  for i in range(3):
    conv_3 = resnet_layer(conv_3,128)

  #conv_4 x6
  conv_4 = resnet_conv_layer(conv_3,256,2)
  for i in range(5):
    conv_4 = resnet_layer(conv_4,256)
  
  #conv_5 x3
  conv_5 = resnet_conv_layer(conv_4,512,2)
  for i in range(2):
    conv_5 = resnet_layer(conv_5,512)
  
  pool = GlobalAvgPool2D()(conv_5)
  output = Dense(number_classes,activation='softmax')(pool)

  model = Model(input_layer,output)

  return model

In [ ]:
def resnet_layer(input_layer,filter):
  
  conv1 = Conv2D(filter,(1,1))(input_layer)
  norm_conv1 = BatchNormalization()(conv1)
  relu_out1 = ReLU()(norm_conv1)

  conv3 = Conv2D(filter,(3,3),padding='same')(relu_out1)
  norm_conv3 = BatchNormalization()(conv3)
  relu_out3 = ReLU()(norm_conv3)

  conv1_2 = Conv2D(4 * filter,(1,1))(relu_out3)
  norm_conv1_2 = BatchNormalization()(conv1_2)
  adding = Add()([ norm_conv1_2 , input_layer ])
  
  output = ReLU()(adding)

  return output

def resnet_conv_layer(input_layer,filter,strides_num):

  conv1 = Conv2D(filter,(1,1))(input_layer)
  norm_conv1 = BatchNormalization()(conv1)
  relu_out1 = ReLU()(norm_conv1)

  conv3 = Conv2D(filter,(3,3),strides = (strides_num,strides_num) ,padding='same')(relu_out1)
  norm_conv3 = BatchNormalization()(conv3)
  relu_out3 = ReLU()(norm_conv3)

  conv1_2 = Conv2D(4 * filter,(1,1))(relu_out3)
  norm_conv1_2 = BatchNormalization()(conv1_2)
  
  extra_conv =  Conv2D(4 * filter,(1,1),strides = (strides_num,strides_num))(input_layer)
  extra_norm =  BatchNormalization()(extra_conv)

  adding = Add()([norm_conv1_2, extra_norm])
  
  output = ReLU()(adding)

  return output

